Importação de bibliotecas

In [12]:
from ucimlrepo import fetch_ucirepo
from sklearn.naive_bayes import GaussianNB
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from collections import Counter
import random
import statistics as st
from sklearn.model_selection import train_test_split


Classe para treinar e criar um classificador bayesiano

In [13]:
class ClassifierTrainer:
    # inicialização da classificação
    def __init__(self, datasets, model):
        self.datasets = datasets
        self.model = model
    
    def build_pipeline(self):
        # função para criar o pipeline
        #esse pipeline consiste em um escalador (StandardScaler) para transformar os valores entre 0 e 1
        # seguido pelo classificador (self.model).
        return Pipeline([
            ('scaler', StandardScaler()),
            ('clf', self.model)
        ])

    def find_majority_and_choose_random(self, row):
        #identifica a classe com a estimativa de maxima verossimilhança
        # se existirem 2 classes com a mesma probabilidade retorna de forma aleatória
        counts = Counter(row)
        max_count = max(counts.values())
        majority_numbers = [num for num, count in counts.items() if count == max_count]
        return random.choice(majority_numbers) if len(majority_numbers) > 1 else majority_numbers[0]

    def find_majority_numbers(self, matrix):
        #as predições foram salvas em uma matriz 
        majority= []
        for row in matrix:
            #cada linha possui a classificação salva em um vetor ex.: [[1],[0]]
            # a função find_majority_and_choose_random recebe uma predição e 
            majority.append(self.find_majority_and_choose_random(row))
        return majority

    # treina o classificador para várias iterações, com diferentes partições do conjunto de dados realizando 30 iterações
    def train_classifiers_with_random_states(self, n_iterations=30):
        #inicia as métricas como vetores vazios
        precision_scores, recall_scores, f1_scores, accuracy_scores = [], [], [], []
        #total de linhas do dataset
        total_rows = len(self.datasets)
        #classes do dataset (estão salvos na coluna diagnosis)
        labels = self.datasets['diagnosis']
        #realiza a divisão em treino e teste e realiza a classificação e validação das métricas 30x
        for _ in tqdm(range(n_iterations)):
            #escolhe os índices aleatoriamente para separar os dados em treino e teste(20% e 80%)
            #além disso utilizei o parâmetro stratify=labels que garante que a distribuição das classes (rótulos) seja similar nos conjuntos de treinamento e teste.
            train_indices, test_indices = train_test_split(np.arange(total_rows), test_size=0.2, random_state=None, stratify=labels)
            #divide o dataset em treino e teste  
            train_data, test_data = self.datasets.iloc[train_indices], self.datasets.iloc[test_indices]
            #divide os dados de treino e teste em X e Y, onde X são os atributos e y a classe 
            #no caso desse projeto a classe é diagnosis podendo ser 0 ou 1 
            X_train, y_train = train_data.drop('diagnosis', axis=1), train_data['diagnosis']
            X_test, y_test = test_data.drop('diagnosis', axis=1), test_data['diagnosis']
            # Escalar os dados
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            
            #chama a função build_pipeline para execução do pipeline, 
            # sequência de etapas de processamento de dados e treinamento de modelo
            pipeline = self.build_pipeline()
            #cálculo das médias e das variancias para cada uma das classes 
            pipeline.fit(X_train, y_train)
            #Realiza a predição para o conjunto de teste.
            y_pred = pipeline.predict(X_test)
            #salva as predições em um vetor 

            #métricas de precision, recall, f1 e acurácia
            precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
            recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
            f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
            accuracy_scores.append(accuracy_score(y_test, y_pred))
        
        self.print_metrics(precision_scores, recall_scores, f1_scores, accuracy_scores, y_test, y_pred)

    def print_metrics(self, precision_scores, recall_scores, f1_scores, accuracy_scores, y_test, y_pred):
        print(f"Precision Mean: {st.mean(precision_scores)}, Std Dev: {st.stdev(precision_scores)}, CI: {np.percentile(precision_scores, [2.5, 97.5])}")
        print(f"Recall Mean: {st.mean(recall_scores)}, Std Dev: {st.stdev(recall_scores)}, CI: {np.percentile(recall_scores, [2.5, 97.5])}")
        print(f"F1 Mean: {st.mean(f1_scores)}, Std Dev: {st.stdev(f1_scores)}, CI: {np.percentile(f1_scores, [2.5, 97.5])}")
        print(f"Accuracy Mean: {st.mean(accuracy_scores)}, Std Dev: {st.stdev(accuracy_scores)}, CI: {np.percentile(accuracy_scores, [2.5, 97.5])}")
        print(classification_report(y_test, y_pred))



Importar os datasets 

In [14]:
spectf_heart = fetch_ucirepo(id=96)

# data (as pandas dataframes)
X = spectf_heart.data.features
y = spectf_heart.data.targets


In [15]:
X.head()

,F1R,F1S,F2R,F2S,F3R,F3S,F4R,F4S,F5R,F5S,...,F18R,F18S,F19R,F19S,F20R,F20S,F21R,F21S,F22R,F22S
0,59,52,70,67,73,66,72,61,58,52,...,66,56,62,56,72,62,74,74,64,67
1,72,62,69,67,78,82,74,65,69,63,...,65,71,63,60,69,73,67,71,56,58
2,71,62,70,64,67,64,79,65,70,69,...,73,70,66,65,64,55,61,41,51,46
3,69,71,70,78,61,63,67,65,59,59,...,61,61,66,65,72,73,68,68,59,63
4,70,66,61,66,61,58,69,69,72,68,...,67,69,70,66,70,64,60,55,49,41


In [16]:
y.head()

,diagnosis
0,1
1,1
2,1
3,1
4,1


In [17]:
df = X.copy()
df['diagnosis'] = y['diagnosis'] 


In [18]:
df.columns

Index(['F1R', 'F1S', 'F2R', 'F2S', 'F3R', 'F3S', 'F4R', 'F4S', 'F5R', 'F5S',
       'F6R', 'F6S', 'F7R', 'F7S', 'F8R', 'F8S', 'F9R', 'F9S', 'F10R', 'F10S',
       'F11R', 'F11S', 'F12R', 'F12S', 'F13R', 'F13S', 'F14R', 'F14S', 'F15R',
       'F15S', 'F16R', 'F16S', 'F17R', 'F17S', 'F18R', 'F18S', 'F19R', 'F19S',
       'F20R', 'F20S', 'F21R', 'F21S', 'F22R', 'F22S', 'diagnosis'],
      dtype='object')

In [19]:
print(len(df))

267


In [20]:
df['diagnosis'].value_counts()

diagnosis
1    212
0     55
Name: count, dtype: int64

In [21]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Classe do Classificador Bayesiano Gaussiano com Covariância Completa
class GaussianBayes:
    def __init__(self):
        self.class_priors = {}  # P(\omega_i)
        self.means = {}         # μ para cada classe
        self.covariances = {}   # Matriz de covariância para cada classe

    def fit(self, X, y):
        """
        Treina o modelo calculando os parâmetros (máxima verossimilhança).
        """
        self.classes = np.unique(y)
        for c in self.classes:
            X_c = X[y == c]  # Dados da classe c
            self.class_priors[c] = len(X_c) / len(X)  # Prior P(\omega_i)
            self.means[c] = X_c.mean(axis=0)  # Vetor de médias para a classe c
            eps = 1e-5  # Regularização
            self.covariances[c] = np.diag(np.var(X_c, axis=0) + eps)

    def _multivariate_gaussian(self, x, mean, covariance):
        """
        Calcula a verossimilhança p(x_k | ω_i) usando a densidade multivariada Gaussiana.
        """
        d = len(x)  # Dimensão do vetor de características
        covariance_det = np.linalg.det(covariance)
        if covariance_det <= 0:  # Verifica problemas de singularidade
            raise ValueError(f"Determinante da matriz de covariância é inválido: {covariance_det}")
        
        covariance_inv = np.linalg.inv(covariance)

        # Termo de normalização
        normalization = (2 * np.pi) ** (-d / 2) * (covariance_det ** -0.5)


        # Termo exponencial
        diff = x - mean
        exponent = -0.5 * diff.T @ covariance_inv @ diff

        return normalization * np.exp(exponent)

    def _posterior(self, x):
        """
        Calcula as probabilidades a posteriori P(\omega_i | x_k) para cada classe.
        """
        posteriors = {}
        for c in self.classes:
            likelihood = self._multivariate_gaussian(x, self.means[c], self.covariances[c])
            posterior = likelihood * self.class_priors[c]
            posteriors[c] = posterior
        return posteriors

    def predict(self, X):
        """
        Prediz a classe para cada exemplo no conjunto X.
        """
        predictions = []
        for x in X:
            posteriors = self._posterior(x)
            predictions.append(max(posteriors, key=posteriors.get))  # Classe com maior P(\omega_i | x_k)
        return np.array(predictions)

In [22]:
# treinando um classificador bayesiano
model = GaussianBayes() 
#inicializando um classificador com a nossa base df e o modelo escolhido 
trainer = ClassifierTrainer(df, model=model)
#realizando o treino e teste nos dados para o classificador trainer, usando 30 partições
trainer.train_classifiers_with_random_states(n_iterations=30)

100%|██████████| 30/30 [00:04<00:00,  7.03it/s]

Precision Mean: 0.8477344164193825, Std Dev: 0.03079161533091068, CI: [0.78776495 0.8907768 ]
Recall Mean: 0.6833333333333333, Std Dev: 0.06119716067219918, CI: [0.57407407 0.77777778]
F1 Mean: 0.7125729421822118, Std Dev: 0.05644638885533776, CI: [0.61152037 0.79818133]
Accuracy Mean: 0.6833333333333333, Std Dev: 0.06119716067219918, CI: [0.57407407 0.77777778]
              precision    recall  f1-score   support

           0       0.40      0.91      0.56        11
           1       0.97      0.65      0.78        43

    accuracy                           0.70        54
   macro avg       0.68      0.78      0.67        54
weighted avg       0.85      0.70      0.73        54

